In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import yaml


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_gwls.txt', 'r') as file:    path_gwls  = file.read()
    
dir_CMIP6    = f'{path_main}Data/CMIP6/HSIs/'
dir_GWL      = f'{path_gwls}cmip6_all_ens/'
dir_EMT      = f'{path_main}Data/CMIP6/EMT/'
dir_names    = f'{path_main}Scripts/Model_lists/'
dir_out      = f'{path_main}Data/Plot_preparation/HWMId/CMIP6/'
if not os.path.exists(dir_out): os.mkdir(dir_out)


## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade', 'Zagreb',
          'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam', 'London', 'Dublin',
          'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk', 'Vilnius', 'Riga', 'Moscow',
          'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

#Define HSIs
HSI_names = ['TX']

#Define models and SSPs which should be used
all_models = dict()
all_models['ssp585'] = []
with open(dir_names + 'Models_CMIP6_SSP585.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['ssp585'].append(line[:-1])
        
#Read warming levels
fname = dir_GWL + 'cmip6_warming_levels_all_ens_1850_1900.yml'
with open(fname, 'r') as file:
    GWL_data = yaml.safe_load(file)
    
#Define warming levels
GWL_levels = ['10', '20', '30']


## Prepare GWL data

In [ ]:
#Define SSP
SSP = 'ssp585'

quantiles = ['Q50', 'Q90']

#Loop over cities
for HSI in HSI_names:
    
    print(HSI)

    #Select file name
    dir_CMIP6_HWMId = dir_CMIP6 + 'HWMId-' + HSI + '/'

    #Create empty array for storing SREX-averaged heat indices
    da_empty1 = np.zeros((len(all_models[SSP]), len(cities), len(GWL_levels), len(quantiles))) * np.NaN
    data_coll = xr.Dataset(coords={'model':     ('model', all_models[SSP]),
                                   'city':      ('city', cities),
                                   'GWL_level': ('GWL_level', GWL_levels),
                                   'quantile': ('quantile', quantiles)})
    data_coll['HWMID'] = (('model', 'city', 'GWL_level', 'quantile'), da_empty1)    

    #Loop over cities
    for city in cities:
        
        #Loop over models
        for i1, model in enumerate(all_models[SSP]):

            #Select ensemble member
            if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'CNRM-CM6-1-HR', 'UKESM1-0-LL', 'MIROC-ES2L']:
                member = "r1i1p1f2"
            elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
                member = "r1i1p1f3"
            else:
                member = "r1i1p1f1"            
            
            #Read data
            fname_CMIP6 = [file for file in os.listdir(dir_CMIP6_HWMId) if HSI in file and SSP + '_' in file and model + '_' in file]
            if len(fname_CMIP6)!=1:  sys.exit('File is not unique')
            data_CMIP6 = xr.open_dataset(dir_CMIP6_HWMId + fname_CMIP6[0])              

            #Read time periods when certain global warming levels (GWL) are reached
            time_GWL = dict()
            for level in GWL_levels:
                data_level = GWL_data['warming_level_' + level]
                entry_sel = [entry for entry in data_level if entry['model']==model and entry['exp']==SSP and entry['ensemble']==member]
                
                #Select data in time period when GWL is reached
                time_GWL = slice(str(entry_sel[0]['start_year']), str(entry_sel[0]['end_year']))
                
                #Calculate median and 90th quantile
                data_Q50 = data_CMIP6.HWMID.sel(city=city, time=time_GWL).median('time')
                data_Q90 = data_CMIP6.HWMID.sel(city=city, time=time_GWL).quantile(0.90, dim='time')
                
                #Put data in array
                data_coll.HWMID.loc[{"model": model, "city": city, "GWL_level": level, "quantile": 'Q50'}] = data_Q50
                data_coll.HWMID.loc[{"model": model, "city": city, "GWL_level": level, "quantile": 'Q90'}] = data_Q90
                
    #Save data in file
    fname_out = dir_out + 'HWMId-' + HSI + '_' + SSP + '_GWL.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)
  

## Prepare data according to European warming (EMT)

In [ ]:
#Define SSP
SSP = 'ssp585'

quantiles = ['Q50', 'Q90']

#Define changes in EMT relative to 1981-2010
EMT_change     = np.array([1.0, 2.0, 3.0])
EMT_change_str = ['1.0K', '2.0K', '3.0K']

#Loop over cities
for HSI in HSI_names:
    
    print(HSI)

    #Select file name
    dir_CMIP6_HWMId = dir_CMIP6 + 'HWMId-' + HSI + '/'

    #Create empty array for storing SREX-averaged heat indices
    da_empty1 = np.zeros((len(all_models[SSP]), len(cities), len(EMT_change_str), len(quantiles))) * np.NaN
    data_coll = xr.Dataset(coords={'model':     ('model', all_models[SSP]),
                                   'city':      ('city', cities),
                                   'EMT_change': ('EMT_change', EMT_change_str),
                                   'quantile': ('quantile', quantiles)})
    data_coll['HWMID'] = (('model', 'city', 'EMT_change', 'quantile'), da_empty1)

    #Loop over cities
    for city in cities:
        
        #Loop over models
        for i1, model in enumerate(all_models[SSP]):

            #Select ensemble member
            if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'CNRM-CM6-1-HR', 'UKESM1-0-LL', 'MIROC-ES2L']:
                member = "r1i1p1f2"
            elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
                member = "r1i1p1f3"
            else:
                member = "r1i1p1f1"            
            
            #Read data
            fname_CMIP6 = [file for file in os.listdir(dir_CMIP6_HWMId) if HSI in file and SSP + '_' in file and model + '_' in file]
            if len(fname_CMIP6)!=1:  sys.exit('File is not unique')
            data_CMIP6 = xr.open_dataset(dir_CMIP6_HWMId + fname_CMIP6[0])              

            #Read European mean temperature (EMT)
            files_EMT = sorted([dir_EMT + file for file in os.listdir(dir_EMT) if model + '_' in file and 'EMT_' in file])
            data_EMT  = xr.concat((xr.open_dataset(file) for file in files_EMT), dim='time')
            
            #Calculate EMT relative to 1981-2010 and calculate 20-year means
            dataEMT_ref = data_EMT.sel(time=slice('1981', '2010')).mean('time')
            dataEMT_rel = data_EMT - dataEMT_ref
            dataEMT_20y = dataEMT_rel.rolling(time=20, center=True).mean()

            #Loop over selected EMT levels
            for dEMT, dEMT_str in zip(EMT_change, EMT_change_str):

                #Identify 20-year period in wich level is reached for first time
                ind  = np.where(dataEMT_20y.tas>dEMT)[0][0]
                central_year = dataEMT_20y.isel(time=ind).time.dt.year
                start_year   = int(central_year - 20 / 2)
                end_year     = int(central_year + (20 / 2 - 1))
                years_sel    = slice(str(start_year), str(end_year))
                
                if end_year>2099:
                    print(model)
                    print(end_year)

                #Calculate median and 90th quantile
                data_Q50 = data_CMIP6.HWMID.sel(city=city, time=years_sel).median('time')
                data_Q90 = data_CMIP6.HWMID.sel(city=city, time=years_sel).quantile(0.90, dim='time')
                
                #Put data in array
                data_coll.HWMID.loc[{"model": model, "city": city, "EMT_change": dEMT_str, "quantile": 'Q50'}] = data_Q50
                data_coll.HWMID.loc[{"model": model, "city": city, "EMT_change": dEMT_str, "quantile": 'Q90'}] = data_Q90
                
    #Save data in file
    fname_out = dir_out + 'HWMId-' + HSI + '_' + SSP + '_EMT.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)
  

## Prepare time period data

In [ ]:
#Select time periods
time_periods = [[1981, 2010],
                [2036, 2065],
                [2070, 2099]]

#Define time string
time_strings = [str(time[0]) + '-' + str(time[1]) for time in time_periods]

#Define SSPs
SSPs = ['ssp585']

quantiles = ['Q50', 'Q90']

#Loop over SSPs
for SSP in SSPs:

    #Loop over cities
    for HSI in HSI_names:
        
        print(HSI)
            
        #Select folder
        dir_CMIP6_HWMId = dir_CMIP6 + 'HWMId-' + HSI + '/'

        #Create empty array for storing SREX-averaged heat indices
        da_empty1 = np.zeros((len(all_models[SSP]), len(cities), len(time_periods), len(quantiles))) * np.NaN
        data_coll = xr.Dataset(coords={'model':       ('model', all_models[SSP]),
                                       'city':        ('city', cities),
                                       'time_period': ('time_period', time_strings),
                                       'quantile': ('quantile', quantiles)})
        data_coll['HWMID'] = (('model', 'city', 'time_period', 'quantile'), da_empty1)    

        #Loop over cities
        for city in cities:
            
            #Loop over models
            for i1, model in enumerate(all_models[SSP]):

                #Read data
                fname_CMIP6 = [file for file in os.listdir(dir_CMIP6_HWMId) if HSI in file and SSP + '_' in file and model + '_' in file]
                if len(fname_CMIP6)!=1:  sys.exit('File is not unique')
                data_CMIP6 = xr.open_dataset(dir_CMIP6_HWMId + fname_CMIP6[0])              

                #Loop over time periods
                for time_period, time_string in zip(time_periods, time_strings):

                    #Select data in time period when GWL is reached
                    time_sel = slice(str(time_period[0]), str(time_period[1]))
                    
                    data_Q50 = data_CMIP6.sel(city=city, time=time_sel).median('time')
                    data_Q90 = data_CMIP6.sel(city=city, time=time_sel).quantile(0.90, dim='time')

                    #Put data in array
                    data_coll.HWMID.loc[{"model": model, "city": city, "time_period": time_string, "quantile": 'Q50'}] = data_Q50.HWMID
                    data_coll.HWMID.loc[{"model": model, "city": city, "time_period": time_string, "quantile": 'Q90'}] = data_Q90.HWMID

        #Save data in file
        fname_out = dir_out + 'HWMId-' + HSI + '_' + SSP + '_time-periods.nc'
        if os.path.exists(fname_out): os.remove(fname_out)
        data_coll.to_netcdf(fname_out)
